## 1. customersOfProfile Function
This notebook contains detailed explanations of the customersOfProfile function provided by the api. These explanations represent the logic behind the functions and show small examples.

### 1.1. Goal
Given a product list and some criteria, return a list of customers which represents a customer profile based on the products.

### 1.2. Function Template
#### Input
localhost:8086/customersOfProfile?jsonData={"Count":5, "MinPercentage":60, "Type": 2, "ProfileId": 123, "ProfileDs": "Bebek", "Products": [{"id": 9556}, {"id": 34398}, {"id": 5974}]}

#### Output
{"Customers": [{"percentage":98, "id": 90361}, {"percentage":80, "id": 90412}, {"percentage":77, "id": 1073258}]}


### 1.3. Logic
In customersOfProfile function, we will use the sales matrix of customers (whose detailed explanation can be found on the description of recommendProducts function). Sales matrix is two-dimensional representation of the sales records whose rows represent customers and columns represent products. For this function, we use level 3 hierarchy of products (IdUrunGrup3 information). Each data point on sales matrix reflects how much money a customer spent for a product group. 

An example sales matrix is shown in Figure 1.1. The shape of sales matrix is (NC x NP) where NC is number of customers and NP is number of products. In this case, NC=6 and NP=4. 

|Customers|P0|P1|P2|P3|
|--|-------------------------------|
|C0 |3 |2 |0 |1 |
|C1 |1 |1 |5 |0 |
|C2 |4 |0 |3 |2 |
|C3 |2 |1 |0 |1 |
|C4 |1 |0 |2 |2 |
|C5 |0 |0 |3 |0 |

<center>Figure 1.1. Sales Matrix</center>

The aim of customersOfProfile function is to find customers who are better explained by the given product list. We are trying to find which customers more fit to the given product list. 

For this purpose, we are using a variation of Nonnegative Matrix Factorization (NMF) model. 

Given a rank (r, which is smaller than the number of products), NMF decomposes original matrix into two smaller sized matrices. The problem can be represented as:

$$ X \approx W H$$

where X matrix's dimensions are (NC x NP), W matrix's dimensions are (NC x r), H matrix's dimensions are (r x NP), NC is the number of customers, NP is the number of products and r is the rank value. The goal of NMF is to generate W and H matrices whose product (WH) is very similar (almost the same) to the original X matrix. 

The algorithm of NMF works as follows:
    
    * Step 0
    W and H matrices are initially created with random values. 
    * Step 1
    W matrix is kept fixed and the values of H matrix is changed in order to make WH matrix more similar to the original X matrix. 
    * Step 2
    Then, H matrix is kept fixed and W is modified for making WH closer to X. 
    * Step 3
    Steps 1 and 2 are applied continueusly until some specific point (specific number of iterations etc). 

The resulting W and H matrices represent the behaviour of original matrix. 

In customersOfProfile function, we are using a variation of NMF model. Instead of randomly generating H matrix and updating it in every iteration, we are keeping it fixed. H matrix represents the characteristics of products (since its shape is (r x NP), each of its column represents a product in a different way). 

    * Step 0
    W matrix is initially created with random values. 
    H matrix is created based on given products.
    * Step 1
    H matrix is kept fixed and W is modified for making WH closer to X. 
    * Step 2
    Step 1 is applied continueusly until some specific point (specific number of iterations etc). 

For this function, we set rank r=1. Assume products P1 and P3 are given to represent a customer profile. The H matrix is generated by assigning 1 to the given products and setting 0 to other products (as shown in Figure 1.2) whose shape is (1 x 4).

|P0|P1|P2|P3|
|--|-------------------------------|
|0 |1 |0 |1 |

<center>Figure 1.2. H Matrix</center>

After generating H matrix, W matrix's entries are updated for several iterations. An example W matrix is shown in Figure 1.3. 

|Customers||
|--|-------------------------------|
|C0 |2 |
|C1 |0 |
|C2 |2 |
|C3 |3 |
|C4 |1 |
|C5 |2 |

<center>Figure 1.3. W Matrix</center>

The multiplication of W and H matrices will be a similar representation of sales matrix based on columns P1 and P3. However, note that since we randomly initialize W matrix, and apply small number of iterations; the results also contain some customers who did not purchase any of given products but expected to buy these products.

After generating W Matrix, we use the formula below to find the percentage value which represents the customer's belonging to the profile. W(C) corresponds to the value of customer C on W Matrix and maxValue corresponds to the maximum value of W Matrix. This formula will set the maximum value as 100% and assign other percentages with fixed ratio. In our simple example, maxValue=3. 

<center>percentage(C) = 100 * W(C) / maxValue</center>

Customers' W Matrix values and the percentages are shown in Figure 1.4. 

|Customers|W Matrix Value| Percentage |
|--|-------------------------------|
|C0 |2 |66 |
|C1 |0 |0 |
|C2 |2 |66 |
|C3 |3 |100 |
|C4 |1 |33 |
|C5 |2 |66 |

<center>Figure 1.4. W Matrix Values and Customer Percentages</center>

The result of customersOfProfile function is the sorted version of above figure. 

|Customers| Percentage |
|--|-------------------------------|
|C3 |100 |
|C0 |66 |
|C2 |66 |
|C5 |66 |
|C4 |33 |
|C1 |0 |

<center>Figure 1.5. Result of customersOfProfile</center>

### 1.4. Count and MinPercentage
Count and MinPercentage parameters filters the customer list found at the end of Section 1.3.

### 1.5. Type

Type parameter defines how the original sales matrix will used during recommendation of the products. 

Type may have the following values: 

|Type|Description|
|--|-------------------------------|
|1|Total Sales Amount|
|2|Whether a sale is done or not (0 or 1. Binary representation)|

<center>Figure 1.6. Type Types</center>

When the criteria is "1", the sales matrix will be used as it is (each entry in the matrix corresponds to the amount of money a customer spent to a product, as demonstrated in section 1.3 of this file). When the criteria is "2", the sales matrix will converted to a binary representation which shows whether a customer bought a product or not, regardless of how much money he/she spent. The binary representation of original sales tensor is shown below. 

|Customers|P0|P1|P2|P3|
|--|-------------------------------|
|C0 |1 |1 |0 |1 |
|C1 |1 |1 |1 |0 |
|C2 |1 |0 |1 |1 |
|C3 |1 |1 |0 |1 |
|C4 |1 |0 |1 |1 |
|C5 |0 |0 |1 |0 |

<center>Figure 1.7. Binary Representation of Sales Matrix</center>

The results of both types may differ dramatically. Intiutively, we can say all products purchased by a customer has similar importance when type=2. However, when type=1, the amount of money spent on a product has importance. 

### 1.6. ProfileId and ProfileDs
After generating the results (customer list and corresponding percentages, filtered by Count and MinPercentage parameters), two tables in the database will be updated.

ProfileId and ProfileDs will be added (or overwritten if previously a customer profile with same id existed) to the ProfileMapping table. 

Customer list and the percentages of customers will be added (or overwritten if previously a customer profile with same id existed) to ProfileCustomers table. 

These tables and customer list will be used in similarCustomers function when the searchType parameter is set to 1.  

### 1.7. Implementational Details
#### 1.7.1. Function Hierarchy
TornadoServer.py file contains main server codes. When a request is sent to the server, it calls the corresponding class. In our case, the class is named as CustomersOfProfile.

First, the class will check the base cases. The base cases in our current implementation are:
    * Checking number of customers
    * Checking minimum percentage
    * Checking type 
    * Checking product list
    
If there is no problem with base cases, the generateCustomerProfile function from ProfileFunctions.py file will be called.

#### generateCustomerProfile
This function fetches the sales matrix, applies modified NMF (which can be foun on NmfFunctions.py file) and updates profile related tables in the database.